In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# import torchvision
# from torch.utils.tensorboard import SummaryWriter

In [2]:
ngpu = 1
# Decide si queremos correr en gpu o cpu
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
device

device(type='cuda', index=0)

In [3]:
ix_to_w = open('word_labels_corenlp.txt', 'r', encoding='utf-8').read().splitlines()
ix_to_w[:6]

[',', 'de', 'que', 'la', 'a', 'el']

In [4]:
w_to_ix = {w: ix for ix, w in enumerate(ix_to_w)}
vocab_size = len(w_to_ix)
vocab_size

7474

In [5]:
class Model(nn.Module):
    def __init__(self, ngpu, D_in, D_emb, D_lstm, D_out):
        super(Model, self).__init__()
        self.ngpu = ngpu
        self.embedding = nn.Embedding(num_embeddings=D_in, embedding_dim=D_emb)#, padding_idx=0)
        self.lstm = nn.LSTM(input_size=D_emb, hidden_size=D_lstm) #, bias=True)#, batch_first=True)
        self.linear = nn.Linear(in_features=D_lstm, out_features=D_out) #, bias=True)

    def forward(self, sentence):
        T = len(sentence)

        embeddings = self.embedding(sentence).view(T, 1, -1)

        lstm_out, (ht, ct) = self.lstm(embeddings)
        lstm_out = lstm_out.view(T, -1)

        preact_out = self.linear(lstm_out).view(T, -1)

        return F.log_softmax(preact_out, dim=1)

In [6]:
# Dimensión de entrada (one-hot), tamaño del vocabulario
D_in = vocab_size

# Dimensión de la capa de embedding
D_emb = 64 # 32

# Dimensión de la capa lstm
D_lstm = 32 # 16

# Dimensión de la capa de salida
D_out = D_in

# Modelo por cargar
model_state = 'modelsaves/modelv2-corenlp-emb_64-lstm_32-seed_42069-epochs_299-best-val'

In [7]:
model = Model(ngpu, D_in, D_emb, D_lstm, D_out)
model.load_state_dict(torch.load(model_state))
model.to(device)
model.eval()

Model(
  (embedding): Embedding(7474, 64)
  (lstm): LSTM(64, 32)
  (linear): Linear(in_features=32, out_features=7474, bias=True)
)

In [8]:
BOS = '<BOS>'
EOS = '<EOS>'
UNK = '<UNK>'

ixBOS = w_to_ix[BOS]
ixEOS = w_to_ix[EOS]
ixUNK = w_to_ix[UNK]

In [9]:
test_sents = [
    'andrés manuel lópez obrador',
    'lópez obrador',
    'ricardo anaya',
    'anaya',
    'josé antonio meade',
    'meade',
    'el candidato',
    '"'
]

In [10]:
test_sents_ix = [
    [ixBOS]+[w_to_ix[w] for w in sent.split()] 
    for sent in test_sents
]
test_sents_ix

[[7471, 64, 56, 29, 30],
 [7471, 29, 30],
 [7471, 49, 41],
 [7471, 41],
 [7471, 48, 50, 34],
 [7471, 34],
 [7471, 5, 26],
 [7471, 10]]

In [11]:
def to_pytorch_tensor(list_of_lists):
    return [
        torch.LongTensor(l).to(device)
        for l in list_of_lists
    ]

In [12]:
X_test = to_pytorch_tensor(test_sents_ix)
X_test

[tensor([7471,   64,   56,   29,   30], device='cuda:0'),
 tensor([7471,   29,   30], device='cuda:0'),
 tensor([7471,   49,   41], device='cuda:0'),
 tensor([7471,   41], device='cuda:0'),
 tensor([7471,   48,   50,   34], device='cuda:0'),
 tensor([7471,   34], device='cuda:0'),
 tensor([7471,    5,   26], device='cuda:0'),
 tensor([7471,   10], device='cuda:0')]

In [13]:
def print_ix_sentence(sentence, end='\n'):
    print(' '.join(ix_to_w[ix] for ix in sentence.data), end=end)

In [14]:
def generar_sent(sentence, limit = 47, choose_max=True):
    if len(sentence) == 0:
        sentence = torch.LongTensor([ixBOS]).to(device)
    torchEOS = torch.LongTensor([ixEOS]).to(device)
    i = 0
    prediccion = sentence[-1:]
    while not torch.eq(prediccion, torchEOS):
        out = model(sentence)
        if choose_max:
            prediccion = torch.argmax(out[-1:], dim=1)
        else:
            prediccion = torch.multinomial(torch.exp(out[-1]), 1)
#         print_ix_sentence(prediccion, end=' ') # Imprimo el siguiente caracter
        sentence = torch.cat((sentence, prediccion), dim=0) # Genero siguiente cadena
        i+=1
        if i>limit:
            break
    return sentence

In [16]:
with torch.no_grad():
    for sentence in X_test[:10]:
        print('===================================================')
        print('- Probando enunciado:')
        print_ix_sentence(sentence)
        print()
        
        print('- Enunciado generado:')
        gen_sentence = generar_sent(sentence, limit=47, choose_max=True)
        print_ix_sentence(gen_sentence)
        print('\n')

- Probando enunciado:
<BOS> andrés manuel lópez obrador

- Enunciado generado:
<BOS> andrés manuel lópez obrador anunció que se <UNK> la elección del país , porque no se <UNK> el dinero de la gente , porque no se <UNK> el dinero de la gente , porque no se <UNK> el dinero de la gente , porque no se <UNK> el dinero de la gente


- Probando enunciado:
<BOS> lópez obrador

- Enunciado generado:
<BOS> lópez obrador expresó que se <UNK> la corrupción , porque no se <UNK> el dinero de la gente , porque no se <UNK> el dinero de la gente , porque no se <UNK> el dinero de la gente , porque no se <UNK> el dinero de la gente , porque


- Probando enunciado:
<BOS> ricardo anaya

- Enunciado generado:
<BOS> ricardo anaya cortés , el candidato de la coalición todos por méxico al frente a la presidencia de la república , josé antonio meade , dijo que se <UNK> la corrupción , porque no se <UNK> el dinero de la gente , porque no se <UNK> el dinero de la


- Probando enunciado:
<BOS> anaya

- Enunciado g

In [15]:
with torch.no_grad():
    for sentence in X_test[:10]:
        print('===================================================')
        print('- Probando enunciado:')
        print_ix_sentence(sentence)
        print()
        
        print('- Enunciado generado:')
        gen_sentence = generar_sent(sentence, limit=47, choose_max=False)
        print_ix_sentence(gen_sentence)
        print('\n')

- Probando enunciado:
<BOS> andrés manuel lópez obrador

- Enunciado generado:
<BOS> andrés manuel lópez obrador llamó a medios de campaña en <UNK> , los jóvenes , en el ine ciro días estado de desarrollo y encabezará zepeda https: mitin para desarrollar el movimiento <UNK> . <EOS>


- Probando enunciado:
<BOS> lópez obrador

- Enunciado generado:
<BOS> lópez obrador respondió para <UNK> la familia . <EOS>


- Probando enunciado:
<BOS> ricardo anaya

- Enunciado generado:
<BOS> ricardo anaya dijo que se han dedicado a lo que mueven el problema de bienes de la asamblea de la semilla histórica . <EOS>


- Probando enunciado:
<BOS> anaya

- Enunciado generado:
<BOS> anaya no a la presidencia de méxico contará a la entidad , el abanderado de los partidos revolucionario institucional ( pri ) , verde ecologista de méxico ( pvem ) y nueva alianza anunció que llega ahora no se necesitan por qué se dieron los sueldos de los estados


- Probando enunciado:
<BOS> josé antonio meade

- Enunciado g